<a href="https://colab.research.google.com/github/Alex-Segarra/data_science_examples/blob/main/GaussianElimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class matrix_solver:

    def __init__(self):

      """
    Matrix Solver solves a system of linear equations using Gaussian elimination.  It has the following methods:

      1. rowechelonform - sorts and solves a matrix for row echelon form.
      2. backsubstitution - uses backsubstitution to solve a matrix once in row echelon form.
      3. gaussianelimination - uses gaussian elimination to solve a matrix.  Calls both rowechelonform and backsubstitution.
    """
      print('Matrix Solver Initialized')

    def sortTransform(self,M, verbose = False):

      """

      Sorts a matrix

      Parameters
      ----------
      M : array
          The matrix to be sorted
      verbose : bool
          Whether to print the matrix operation after key step

      Returns
      -------
      newM : array
      """

      newM = M.copy()

      for k in reversed(range(newM.shape[1])):
          # Create a boolean matrix, where 1 is values above 0.
          boolM = np.where(np.isclose(newM, 0), 0 ,1)
          # Sort the matrix by the boolean matrix, this preserves the order as best as possible, only switching the rows when there is a non-zero value below a zero.
          newIndeces = np.argsort(-boolM[:,k])
          # Sort the matrix
          newM = newM[newIndeces]

      return newM

    def rowOperationTransform(self,M, verbose = False):

        """

        Sorts and solves a matrix for row echelon form

        Parameters
        ----------
        M : array
            The matrix to be sorted and solved for row echelon form
        verbose : bool
            Whether to print the matrix operation after key step

        Returns
        -------
        newM : array
        """

        if M.shape[0] == 1 and M.shape[1] == 1:
            print(f"\nMatrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            M = M / M if M > 1 else M
            return M

        elif M.shape[0] == 0 or M.shape[1] == 0:
            print(f"\nMatrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            return M

        elif np.all(M==0):
            print(f"\nMatrix is is all zeros") if verbose == True else None
            return M

        else:

            #Sort Matrix
            A = sortTransform(M) if np.nonzero(M[:,0])[0].size < len(M[:,0]) else M.copy()
            print(f"\nSorting Matrix \n {str(A)}") if verbose == True else None
            print("*"*75) if verbose == True else None
            # Divide row by its first non-zero element
            for row in range(M.shape[0]):
                print(f"\nDividing row {row} ({A[row]}) by {A[row,0]} to get {A[row] / A[row,0] if A[row,0] else A[row]}") if verbose == True else None
                A[row] = A[row] / A[row,0] if A[row,0] else A[row]
                print(f"\nThe result is {A}") if verbose == True else None
                print("*"*75) if verbose == True else None
            # Subtract the row from all other rows
            A[1:] -= A[0] * A[1:,0:1]
            print(f"\nSubtracting row {0} from all other rows.\n the result is\n {str(A)}") if verbose == True else None
            print("*"*75) if verbose == True else None
            # Check if there are any other non-zero elements below the first row
            k = 1 if np.nonzero(A[:,0])[0].size > 0 else 0
            # If there are, repeat the process by taking a subset of the matrix, k rows below the first (determined by whether there are non-zero elements below the first row)
            B = A[k:,1:]
            print(f"\nSending the subset of the matrix\n {str(B)}") if verbose == True else None
            print("*"*75) if verbose == True else None
            # Recursively call the function
            B = self.rowOperationTransform(B)
            # Update the matrix
            A[k:,1:] = B
            print(f"\nThe resulting matrix is\n {str(A)}") if verbose == True else None
            print("*"*75) if verbose == True else None

            return A

    def backSubstitution(self, M, verbose = False):

        """
        Uses backsubstitution to solve a matrix once in row echelon form

        Parameters
        ----------
        M : array
            The matrix in row echelon form
        verbose : bool
            Whether to print the matrix operation after key step

        Returns
        -------
        newM : array
            The matrix in row echelon form
        """

        A = M.copy()
        numrows = A.shape[0]
        if not numrows == 1:
            #
            for row in reversed(range(numrows-1)):

                print("*"*75) if verbose == True else None

                col_index = np.nonzero(A[-1,:])[0][0]

                value = M[row,col_index]

                print(f"\nWe multiply {value} against {M[-1]} to get {value * M[-1]}") if verbose == True else None

                print(f"\nWe substract {value * M[-1]} from {M[row]}") if verbose == True else None

                A[row] = M[row] - value * M[-1] # Using M to preserve initial value

                print(f"\nThe result is: {A[row]}") if verbose == True else None

            B = A[:-1,:]

            print("*"*75) if verbose == True else None

            print(f"\nSending this matrix into recursion: \n{B}\n...to restart the process and assess against a new pivot.") if verbose == True else None

            A[:-1,:] = self.backSubstitution(B,verbose=verbose)

        else:
            print('\nReached first row') if verbose == True else None
            return A

        #final check

        return A

    def gaussianelimination(self, M,verbose=False):

        """
        Uses gaussian elimination to solve a matrix

        Parameters
        ----------
        M : array
            The matrix to be solved
        verbose : bool
            Whether to print the matrix operation after key step

        Returns
        -------
        newM : array
            The matrix in row echelon form
        """
        print(f"\nMatrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
        A = M.copy().astype('float')
        n = np.min([A.shape[0], A.shape[1]])

        if np.linalg.det(M[:n,:n]) == 0:
            print('Singular Matrix')
            return A
        else:
            A = self.rowOperationTransform(A,verbose=verbose)
            if not isinstance(A,str):
                A = self.backSubstitution(A,verbose=verbose)
            return A


In [ ]:
M = np.array([[1,2,3,7],[6,5,6,1],[7,2,9,3]])

In [ ]:
solver = matrix_solver()

Matrix Solver Initialized


In [ ]:
solver.gaussianelimination(M, verbose=True)


Matrix is 3x4

Sorting Matrix 
 [[1. 2. 3. 7.]
 [6. 5. 6. 1.]
 [7. 2. 9. 3.]]
***************************************************************************

Dividing row 0 ([1. 2. 3. 7.]) by 1.0 to get [1. 2. 3. 7.]

The result is [[1. 2. 3. 7.]
 [6. 5. 6. 1.]
 [7. 2. 9. 3.]]
***************************************************************************

Dividing row 1 ([6. 5. 6. 1.]) by 6.0 to get [1.         0.83333333 1.         0.16666667]

The result is [[1.         2.         3.         7.        ]
 [1.         0.83333333 1.         0.16666667]
 [7.         2.         9.         3.        ]]
***************************************************************************

Dividing row 2 ([7. 2. 9. 3.]) by 7.0 to get [1.         0.28571429 1.28571429 0.42857143]

The result is [[1.         2.         3.         7.        ]
 [1.         0.83333333 1.         0.16666667]
 [1.         0.28571429 1.28571429 0.42857143]]
**************************************************************************

array([[ 1.        ,  0.        ,  0.        , -3.5       ],
       [ 0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  1.        ,  2.83333333]])